# Infomation Annotation
Trích xuất thông tin từ annotation

Thêm thư viện

In [12]:
from lib_all import *
from data.path import get_path
from config import get_config

In [13]:
config = get_config()

Xây dựng lớp InfoAnno
- Output: các thông tin về bounding box và label
- Kế thừa từ object

In [14]:
class InfoAnno(object): # Kế thừa từ object
    #constructor
    def __init__(self, classes): #21 cái class của VOC
        self.classes = classes

    def __call__(self, anno_path, width, height):
        """
        :param anno_path: [1] đường link dẫn tới xml
        :param width: [any] Chiều rộng của ảnh 
        :param height: [any] Chiều cao của ảnh 
        :return: [[xmin, ymin, xmax, ymax, labelid],...]
        """
        #chứa annotation của ảnh
        list_annotation = []

        # đọc file này lên
        xml = ET.parse(anno_path).getroot()

        for obj in xml.iter('object'): #Lấy ra từng object
            # độ khó
            difficult = int(obj.find('difficult').text)
            #khó quá bỏ qua
            if difficult == 1:
                continue

            # lấy bnb box
            annotation_obj = []
            bndbox = obj.find("bndbox")
            points = ['xmin', 'ymin', 'xmax', 'ymax']
            for pt in points:
                pixel = int(bndbox.find(pt).text) - 1

                #đưa về dạng tỷ lệ
                if pt == 'xmin' or pt  == 'xmax':
                    pixel /= width
                else:
                    pixel /=height

                annotation_obj.append(pixel)

            name = obj.find("name").text.lower().strip()
            # đưa vào trong class chứa các nhãn của tập VOC =>  id của label đấy
            label_id =self.classes.index(name)
            annotation_obj.append(label_id)

            list_annotation += [annotation_obj]
        return np.array(list_annotation)

# Test

In [ ]:
root_path = './data_set/VOCdevkit/VOC2012/'
train_images_path, train_annotations_path, val_images_path, val_annotations_path = get_path(root_path)

Thông số trên từng ảnh

In [19]:
img = cv2.imread(val_images_path[1]) #[height, width, 3 channels BGR]
height, width, channels = img.shape
print(img.shape)

(333, 500, 3)


In [16]:
anno_xml = InfoAnno(config["classes"]) # [20]
inform_annotation = anno_xml(val_annotations_path[1], width, height)
print(inform_annotation)

[[ 0.09        0.03003003  0.998       0.996997   18.        ]
 [ 0.122       0.56756757  0.164       0.72672673 14.        ]]
